In [1]:
import sys
import os
if not sys.modules.get('selenium'):
    os.system('pip3 install selenium')
if not sys.modules.get('pandas'):
    os.system('pip3 install pandas')
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
# expected_conditions는 Selenium 2.26.0 이후 부터 사용 가능합니다.
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import time
import pandas as pd
from selenium.webdriver.common.keys import Keys

In [2]:
# 유튜브 광고제거를 위한 ublock 옵션 추가
chop = webdriver.ChromeOptions()
chop.add_extension('uBlock-Origin-development-build_v1.20.3.102.crx')
driver = webdriver.Chrome(chrome_options = chop)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: use options instead of chrome_options
  after removing the cwd from sys.path.


In [3]:
def save_csv(jungbo, code):      
    frame = pd.DataFrame(jungbo)
    with open('./data/youtube_{0}.txt'.format(code), 'a',encoding='utf-8') as f:
        frame.to_csv(f, header=False, index=False, encoding='utf-8')

In [4]:
## 'text', '조회수', '구독수', '좋아요', '나빠요', '태그'에 대한 내용을 저장하는 방법

def save_csv2(jungbo, code):      
    frame = pd.DataFrame(jungbo)
    with open('./data/youtube_{0}2.txt'.format(code), 'a',encoding='utf-8') as f:
        frame.to_csv(f, header=False, index=False, encoding='utf-8')

In [5]:
def start_crawler():
    #유튜브 열고 로그인화면으로 이동
    driver.get('https://www.youtube.com/?gl=KR&hl=ko')
    time.sleep(3)
    # 검색란을 'engine'으로 입력
    engine = WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.CSS_SELECTOR,'#search')))
    engine.click()
    # 엔진 칸에 '타이레놀'을 입력
    engine.send_keys('타이레놀')
    # 드라이버 엔진 클릭
    driver.find_element_by_css_selector('#search-icon-legacy').click()
    time.sleep(3)

def open_filter(code):
    a = WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.CSS_SELECTOR, '#page-manager > ytd-search')))
    time.sleep(2)
    b = a.find_elements_by_css_selector('#button')[0].click()
    time.sleep(1)

In [6]:
# open_filter(fil)

In [7]:
# order_video(fil)

In [8]:
#page-manager > ytd-search
#dismissable
#dismissable

In [9]:
# 구두점 제거 함수 생성

def remove_punctuation(code):

    sms_no_punctuation = [ch for ch in code if ch not in string.punctuation]
    sms_no_punctuation = "".join(sms_no_punctuation)
        
    return sms_no_punctuation

In [10]:
import string

In [11]:
# 위에서부터 몇 번째 동영상인지를 나타내는 것을 start_code로 잡음
# while문 돌리기 전에 미리 지정을 해줘야한다.

start_code = 0

In [12]:
def open_video(start_code):
    # 동영상이 있는 큰 부분을 op로 잡았다.
    op = WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.CSS_SELECTOR,'#page-manager > ytd-search')))
    start_code = 0
    while start_code < 10:
        # op에서 start_code번째에 존재하는 동영상을 클릭하여 재생시킬것이다.
        f = op.find_elements_by_css_selector('#dismissable')[start_code]
        f.click()
        time.sleep(2)
        # 유투브에서 ...이 존재하는 큰 부분을 g로 잡았다.
        g = WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.CSS_SELECTOR,'#info-contents > ytd-video-primary-info-renderer')))
        time.sleep(1)
        # g부분에서 ...부분을 클릭한다.
        h = g.find_elements_by_css_selector('#button > yt-icon')[4]
        h.click()
        time.sleep(1)
        start_code = start_code + 1
        
        try : 
            # code를 구두점을 제거한 유투브 영상 제목으로 지정
            code = ""
            name = WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.CSS_SELECTOR,'#container > h1 > yt-formatted-string')))
            code = remove_punctuation(name.text)
            # 자막버튼을 찾는다.
            driver.find_element_by_css_selector('#items > ytd-menu-service-item-renderer:nth-child(2) > paper-item > yt-formatted-string').click()
            data = []
            # 자막이 있다면 자막을 저장한다.
            c = WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.CSS_SELECTOR,'#body > ytd-transcript-body-renderer')))
            data.append(c.text)
            save_csv(data,code)
            # 자막변환이 없으면 그냥 패스
            
            # 영상아래부분을 mo로 잡고 그 중에서 '더보기'버튼을 mor로 잡고 클릭하였다.
            # data2는 빈 데이터로 잡고 계속해서 얻은 문구들을 넣을 것이다.
            mo = WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.CSS_SELECTOR,'#meta-contents > ytd-video-secondary-info-renderer')))
            mor = mo.find_elements_by_css_selector('#more > yt-formatted-string')[0].click()
            data2 = []
            # '더보기'를 눌려서 얻을 수 있는 모든 텍스트들을 te로 잡고 .text를 사용하여 문자열로 바꾼 후 tes로 잡는다.
            te = WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.CSS_SELECTOR,'#description > yt-formatted-string')))
            tes = te.text
            data2.append(tes)
            data2.append("----------------------")
            # 조회수를 num으로 잡는다.
            num = WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.CSS_SELECTOR,'#count > yt-view-count-renderer > span.view-count.style-scope.yt-view-count-renderer')))
            data2.append(num.text)
            data2.append("----------------------")
            # 구독자수를 mod로 잡는다.
            mod = mo.find_elements_by_css_selector('#text')[0]
            data2.append(mod.text)
            data2.append("----------------------")
            # 아까 g로 잡았던 부분에서 좋아요는 good으로 잡고, 나빠요는 bad으로 잡아서 추가시킨다.
            g = WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.CSS_SELECTOR,'#info-contents > ytd-video-primary-info-renderer')))
            good = g.find_elements_by_css_selector('#top-level-buttons > ytd-toggle-button-renderer:nth-child(1) > a')[0]
            goods = '좋아요 : ' + good.text
            data2.append(goods)
            bad = g.find_elements_by_css_selector('#top-level-buttons > ytd-toggle-button-renderer:nth-child(2) > a')[0]
            bads = '나빠요 : ' + bad.text
            data2.append(bads)
            data2.append("----------------------")
            # 태크가 존재하면 g에서 tag를 불러들이고, 없으면 pass하기 위해서 try~escept를 사용했다.
            try:
                tag = g.find_elements_by_css_selector('#container > yt-formatted-string')[0]
                data2.append(tag.text)
            except Exception as e:
                time.sleep(1)
                pass
            save_csv2(data2, code)
            
            
            driver.back()
        except Exception as e: 
            time.sleep(2)
            driver.back()
            pass
        if start_code == 10:
            print('완료')

In [13]:
#meta-contents > ytd-video-secondary-info-renderer
#more > yt-formatted-string
#info-contents > ytd-video-primary-info-renderer

num = WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.CSS_SELECTOR,'#count > yt-view-count-renderer > span.view-count.style-scope.yt-view-count-renderer')))
mo = WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.CSS_SELECTOR,'#meta-contents > ytd-video-secondary-info-renderer')))
mor = mo.find_elements_by_css_selector('#more > yt-formatted-string')[0].click()
data2 = []

num = WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.CSS_SELECTOR,'#count > yt-view-count-renderer > span.view-count.style-scope.yt-view-count-renderer')))
num.text

mo = WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.CSS_SELECTOR,'#meta-contents > ytd-video-secondary-info-renderer')))
mor = mo.find_elements_by_css_selector('#more > yt-formatted-string')[0].click()

te = WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.CSS_SELECTOR,'#description > yt-formatted-string')))
te.text

mor = mo.find_elements_by_css_selector('#text')[0]
mor.text

In [14]:
#info-contents > ytd-video-primary-info-renderer
#top-level-buttons > ytd-toggle-button-renderer:nth-child(1) > a
#top-level-buttons > ytd-toggle-button-renderer:nth-child(2) > a

g = WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.CSS_SELECTOR,'#info-contents > ytd-video-primary-info-renderer')))
good = g.find_elements_by_css_selector('#top-level-buttons > ytd-toggle-button-renderer:nth-child(1) > a')[0]
good = print('좋아요 :',good.text)

bad = g.find_elements_by_css_selector('#top-level-buttons > ytd-toggle-button-renderer:nth-child(2) > a')[0]
bad = print('나빠요 :',bad.text)

tag = g.find_elements_by_css_selector('#container > yt-formatted-string')[0]
tag.text

#container > yt-formatted-string

In [15]:
#header > ytd-comments-header-renderer

co = WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.CSS_SELECTOR,'#header > ytd-comments-header-renderer')))
comme = co.find_elements_by_css_selector('#count > yt-formatted-string')[0]
comme.text

In [16]:
#main
#main
#sections
#primary

try:
    com = WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.CSS_SELECTOR,'#sections')))
    com1 = com.find_elements_by_css_selector('#content-text')[0].text
    com2 = com.find_elements_by_css_selector('#content-text')[1].text
    com3 = com.find_elements_by_css_selector('#content-text')[2].text
except Exception as e:
    time.sleep(2)
    pass

mo = WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.CSS_SELECTOR,'#meta-contents > ytd-video-secondary-info-renderer')))
mor = mo.find_elements_by_css_selector('#more > yt-formatted-string')[0].click()

code = ""
name = WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.CSS_SELECTOR,'#container > h1 > yt-formatted-string')))
code = remove_punctuation(name.text)


mo = WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.CSS_SELECTOR,'#meta-contents > ytd-video-secondary-info-renderer')))
mor = mo.find_elements_by_css_selector('#more > yt-formatted-string')[0].click()
data2 = []
te = WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.CSS_SELECTOR,'#description > yt-formatted-string')))
tes = te.text
data2.append(tes)
data2.append("----------------------")
num = WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.CSS_SELECTOR,'#count > yt-view-count-renderer > span.view-count.style-scope.yt-view-count-renderer')))
data2.append(num.text)
data2.append("----------------------")
mor = mo.find_elements_by_css_selector('#text')[0]
data2.append(mor.text)
data2.append("----------------------")
g = WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.CSS_SELECTOR,'#info-contents > ytd-video-primary-info-renderer')))
good = g.find_elements_by_css_selector('#top-level-buttons > ytd-toggle-button-renderer:nth-child(1) > a')[0]
goods = '좋아요 : ' + good.text
data2.append(goods)
bad = g.find_elements_by_css_selector('#top-level-buttons > ytd-toggle-button-renderer:nth-child(2) > a')[0]
bads = '나빠요 : ' + bad.text
data2.append(bads)
data2.append("----------------------")
tag = g.find_elements_by_css_selector('#container > yt-formatted-string')[0]
data2.append(tag.text)
data2.append("----------------------")
time.sleep(3)
co = WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.CSS_SELECTOR,'#header > ytd-comments-header-renderer')))
comme = co.find_elements_by_css_selector('#count > yt-formatted-string')[0]
data2.append(comme.text)
data2.append("----------------------")

try:
    com = WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.CSS_SELECTOR,'#sections')))
    com1 = com.find_elements_by_css_selector('#content-text')[0].text
    data2.append(com1)
    com2 = com.find_elements_by_css_selector('#content-text')[1].text
    data2.append(com2)
    com3 = com.find_elements_by_css_selector('#content-text')[2].text
    data2.append(com3)
except Exception as e:
    time.sleep(2)
    pass

save_csv2(data2, code)


In [17]:
#description > yt-formatted-string

In [18]:
start_crawler()

open_video(start_code)

완료


In [19]:
#info-contents > ytd-video-primary-info-renderer
#container > h1 > yt-formatted-string
#body > ytd-transcript-body-renderer